# [AI模型]

In [1]:
from torch import nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## LSTM

In [2]:
class LSTM(nn.Module):  # LSTM: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.num_directions = 1
        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(
            self.input_size,  # 餵入的特徵種數 (ex:2個變量, input_szie=2)
            self.hidden_size, 
            self.num_layers, 
            batch_first=True  # Default:(seq_len, batch_size, hidden_size)； True:(batch_size, seq_len, hidden_size)
        )
        self.linear = nn.Linear(self.hidden_size, self.output_size)  # 串接:(hidden_size) -> (1)

    def forward(self, input_seq):
        batch_size, seq_len = input_seq.shape[0], input_seq.shape[1]  # 0:batch_size, 1:seq_len
        h_0 = torch.randn(self.num_directions * self.num_layers, batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, batch_size, self.hidden_size).to(device)
        # print(input_seq.size())
        # input(batch_size, seq_len, input_size)
        # input_seq = input_seq.view(self.batch_size, seq_len, self.input_size)
        # output(batch_size, seq_len, num_directions * hidden_size)
        output, _ = self.lstm(input_seq, (h_0, c_0))  # model_out:(batch_size, seq_len, hidden_size)
        # print('output.size=', output.size())
        # print(self.batch_size * seq_len, self.hidden_size)
        # output = output.contiguous().view(batch_size * seq_len, self.hidden_size)  # (5 * 30, 64)
        pred = self.linear(output)  # pred(batch_size, seq_len, output_size)  # 全連接層
        # print('pred=', pred.shape)
        # pred = pred.view(batch_size, seq_len, -1)
        pred = pred[:, -1, :]

        return pred

## RNN

In [3]:
class Simple_RNN(nn.Module):  # LSTM: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.num_directions = 1
        self.batch_size = batch_size
        
        self.rnn = nn.RNN(
            self.input_size,  # 餵入的特徵種數 (ex:2個變量, input_szie=2)
            self.hidden_size, 
            self.num_layers, 
            batch_first=True  # Default:(seq_len, batch_size, hidden_size)； True:(batch_size, seq_len, hidden_size)
        )
        self.linear = nn.Linear(self.hidden_size, self.output_size)  # 串接:(hidden_size) -> (1)

    def forward(self, input_seq):
        batch_size, seq_len = input_seq.shape[0], input_seq.shape[1]  # 0:batch_size, 1:seq_len
        h_0 = torch.randn(self.num_directions * self.num_layers, batch_size, self.hidden_size).to(device)
        # print(input_seq.size())
        # input(batch_size, seq_len, input_size)
        # input_seq = input_seq.view(self.batch_size, seq_len, self.input_size)
        # output(batch_size, seq_len, num_directions * hidden_size)
        output, _ = self.rnn(input_seq, (h_0))  # model_out:(batch_size, seq_len, hidden_size)
        # print('output.size=', output.size())
        # print(self.batch_size * seq_len, self.hidden_size)
        # output = output.contiguous().view(batch_size * seq_len, self.hidden_size)  # (5 * 30, 64)
        pred = self.linear(output)  # pred(batch_size, seq_len, output_size)  # 全連接層
        # print('pred=', pred.shape)
        # pred = pred.view(batch_size, seq_len, -1)
        pred = pred[:, -1, :]

        return pred

## TCN

In [ ]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.conv_layers = []  # 存儲卷積層的列表
        self.num_layers = len(num_channels)  # 卷積層的數量
        for i in range(self.num_layers):
            dilation_size = 2 ** i  # 膨脹率
            in_channels = self.input_size if i == 0 else self.num_channels[i-1]   # 輸入通道數
            out_channels = self.num_channels[i]  # 輸出通道數
            padding_size = int((self.kernel_size - 1) * dilation_size / 2)  # padding 大小
            conv_layer = nn.Conv1d(in_channels, out_channels, self.kernel_size, dilation=dilation_size, padding=padding_size)  # 定義卷積層
            self.conv_layers.append(conv_layer)  # 添加到卷積層列表中
        self.conv_layers = nn.ModuleList(self.conv_layers)  # 將卷積層列表轉換成ModuleList
        self.dropout_layer = nn.Dropout(self.dropout)  # 定義 Dropout層
        self.fc_layer = nn.Linear(self.num_channels[-1], self.output_size)  # 定義全連接層

    def forward(self, x):
        """
        [需做shape的轉換]
        len(x): batch_size
        len(x[0]): seq_length
        len(x[0][0]): input_size (input_channel)
        """
        # shape: (N, L_in, C_in) 
        output = x.transpose(1,2)  # 輸入張量-轉置
        # shape: (N, C_in, L_in)
        
        """Inputs have to have dimension (N, C_in, L_in)
        解釋: N 表示批次大小（batch size），C_in 表示序列長度（number of input channels），L_in 表示序列长度（sequence length）
        """
        for i in range(self.num_layers):
            conv_layer = self.conv_layers[i]  # 取出卷積層
            output = conv_layer(output)  # 卷積操作
            output = nn.functional.relu(output)  # ReLU 激活函數
            output = self.dropout_layer(output)  # Dropout 操作
        output = output.transpose(1,2)  # 輸出張量-轉置
        
        output = self.fc_layer(output[:,-1,:])   # 全連接層
        return output

In [ ]:
import torch
import torch.nn as nn

class TCN_LSTM(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, hidden_size, num_layers):
        super(TCN_LSTM, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.conv_layers = []
        self.num_layers = len(num_channels)
        for i in range(self.num_layers):
            dilation_size = 2 ** i
            in_channels = self.input_size if i == 0 else self.num_channels[i-1]
            out_channels = self.num_channels[i]
            padding_size = int((self.kernel_size - 1) * dilation_size / 2)
            conv_layer = nn.Conv1d(in_channels, out_channels, self.kernel_size, dilation=dilation_size, padding=padding_size)
            self.conv_layers.append(conv_layer)
        self.conv_layers = nn.ModuleList(self.conv_layers)
        self.dropout_layer = nn.Dropout(self.dropout)
        self.lstm_layer = nn.LSTM(self.num_channels[-1], hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_layer = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        output = x.transpose(1,2)
        for i in range(self.num_layers):
            conv_layer = self.conv_layers[i]
            output = conv_layer(output)
            output = nn.functional.relu(output)
            output = self.dropout_layer(output)
        output = output.transpose(1,2)
        output, _ = self.lstm_layer(output)
        output = self.fc_layer(output[:,-1,:])
        return output